In [73]:
# Code based on https://gist.github.com/Spaxe/94e130c73a1b835d3c30ea672ec7e5fe#file-csv-to-geojson-ipynb

In [74]:
import pandas
import json

In [75]:
import pandas as pd
import shutil, os, glob
import pymongo
from pymongo import MongoClient
import json
import schedule
import time
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [76]:
df = pd.concat(map(pd.read_csv, glob.glob('input' + '\*.csv')))
#Extract All source code
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [77]:
df.head()

,Unnamed: 0,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,cluster
0,82,03/06/1965,11:10:51,-18.425,-132.923,Earthquake,10.0,NaN,NaN,5.6,...,NaN,NaN,NaN,NaN,ISCGEM858385,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
1,123,04/16/1965,23:22:21,64.572,-160.375,Earthquake,15.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,ISCGEM857809,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
2,136,04/29/1965,15:28:45,47.288,-122.406,Earthquake,64.7,NaN,NaN,6.7,...,NaN,NaN,NaN,NaN,ISCGEM858143,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
3,179,06/17/1965,20:14:51,32.052,87.784,Earthquake,10.0,NaN,NaN,5.9,...,NaN,NaN,NaN,NaN,ISCGEM856219,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
4,252,09/09/1965,10:02:26,6.470,-84.408,Earthquake,15.0,NaN,NaN,6.2,...,NaN,NaN,NaN,NaN,ISCGEM853712,ISCGEM,ISCGEM,ISCGEM,Automatic,-1


In [78]:
df.dtypes

Unnamed: 0                      int64
Date                           object
Time                           object
Latitude                      float64
Longitude                     float64
Type                           object
Depth                         float64
Depth Error                   float64
Depth Seismic Stations        float64
Magnitude                     float64
Magnitude Type                 object
Magnitude Error               float64
Magnitude Seismic Stations    float64
Azimuthal Gap                 float64
Horizontal Distance           float64
Horizontal Error              float64
Root Mean Square              float64
ID                             object
Source                         object
Location Source                object
Magnitude Source               object
Status                         object
cluster                         int64
dtype: object

In [79]:
# Before renaming the columns 
print(df.columns) 


Index(['Unnamed: 0', 'Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth',
       'Depth Error', 'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status', 'cluster'],
      dtype='object')


In [80]:

df.rename(columns = {'Latitude':'lat', 'Longitude':'lng'}, inplace = True) 
   
# After renaming the columns 
print(df.columns) 

Index(['Unnamed: 0', 'Date', 'Time', 'lat', 'lng', 'Type', 'Depth',
       'Depth Error', 'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status', 'cluster'],
      dtype='object')


In [81]:
df.head()

,Unnamed: 0,Date,Time,lat,lng,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,cluster
0,82,03/06/1965,11:10:51,-18.425,-132.923,Earthquake,10.0,NaN,NaN,5.6,...,NaN,NaN,NaN,NaN,ISCGEM858385,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
1,123,04/16/1965,23:22:21,64.572,-160.375,Earthquake,15.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,ISCGEM857809,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
2,136,04/29/1965,15:28:45,47.288,-122.406,Earthquake,64.7,NaN,NaN,6.7,...,NaN,NaN,NaN,NaN,ISCGEM858143,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
3,179,06/17/1965,20:14:51,32.052,87.784,Earthquake,10.0,NaN,NaN,5.9,...,NaN,NaN,NaN,NaN,ISCGEM856219,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
4,252,09/09/1965,10:02:26,6.470,-84.408,Earthquake,15.0,NaN,NaN,6.2,...,NaN,NaN,NaN,NaN,ISCGEM853712,ISCGEM,ISCGEM,ISCGEM,Automatic,-1


In [82]:
earthquakes = df

In [83]:
earthquakes.head()

,Unnamed: 0,Date,Time,lat,lng,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,...,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,cluster
0,82,03/06/1965,11:10:51,-18.425,-132.923,Earthquake,10.0,NaN,NaN,5.6,...,NaN,NaN,NaN,NaN,ISCGEM858385,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
1,123,04/16/1965,23:22:21,64.572,-160.375,Earthquake,15.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,ISCGEM857809,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
2,136,04/29/1965,15:28:45,47.288,-122.406,Earthquake,64.7,NaN,NaN,6.7,...,NaN,NaN,NaN,NaN,ISCGEM858143,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
3,179,06/17/1965,20:14:51,32.052,87.784,Earthquake,10.0,NaN,NaN,5.9,...,NaN,NaN,NaN,NaN,ISCGEM856219,ISCGEM,ISCGEM,ISCGEM,Automatic,-1
4,252,09/09/1965,10:02:26,6.470,-84.408,Earthquake,15.0,NaN,NaN,6.2,...,NaN,NaN,NaN,NaN,ISCGEM853712,ISCGEM,ISCGEM,ISCGEM,Automatic,-1


In [84]:
earthquakes['city'] = ""
earthquakes['country'] = ""
count = 0
for index, row in earthquakes.iterrows():
   near_city = citipy.nearest_city(row['lat'], row['lng']).city_name
   near_country = citipy.nearest_city(row['lat'], row['lng']).country_code
   earthquakes.set_value(index,"city",near_city)
   earthquakes.set_value(index,"country",near_country)
earthquakes.head()

C:\Users\mwals\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
C:\Users\mwals\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,Unnamed: 0,Date,Time,lat,lng,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,...,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status,cluster,city,country
0,82,03/06/1965,11:10:51,-18.425,-132.923,Earthquake,10.0,NaN,NaN,5.6,...,NaN,NaN,ISCGEM858385,ISCGEM,ISCGEM,ISCGEM,Automatic,-1,rikitea,pf
1,123,04/16/1965,23:22:21,64.572,-160.375,Earthquake,15.0,NaN,NaN,6.0,...,NaN,NaN,ISCGEM857809,ISCGEM,ISCGEM,ISCGEM,Automatic,-1,bethel,us
2,136,04/29/1965,15:28:45,47.288,-122.406,Earthquake,64.7,NaN,NaN,6.7,...,NaN,NaN,ISCGEM858143,ISCGEM,ISCGEM,ISCGEM,Automatic,-1,tacoma,us
3,179,06/17/1965,20:14:51,32.052,87.784,Earthquake,10.0,NaN,NaN,5.9,...,NaN,NaN,ISCGEM856219,ISCGEM,ISCGEM,ISCGEM,Automatic,-1,lasa,cn
4,252,09/09/1965,10:02:26,6.470,-84.408,Earthquake,15.0,NaN,NaN,6.2,...,NaN,NaN,ISCGEM853712,ISCGEM,ISCGEM,ISCGEM,Automatic,-1,burica,pa


In [85]:
json_result_string = df.to_json(
    orient='records', 
    double_precision=12,
    date_format='iso'
)
json_result = json.loads(json_result_string)

In [68]:
geojson = {
    'type': 'FeatureCollection',
    'features': []
}
for record in json_result:
    geojson['features'].append({
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [record['lng'], record['lat']],
        },
        'properties': record,
    })

In [86]:
# Writing converted data back to input folder
with open('input/earthquakes-dbscan-anomaly-processed_geoJSON_fmt.json', 'w') as f:
    f.write(json.dumps(geojson, indent=2))